In [0]:
# 사용할 라이브러리를 불러옵니다.
import io                                              # 입력(input)과 출력(output) 관련 라이브러리입니다.
import numpy as np                                     # 배열(array)을 쉽게 사용하기 위한 라이브러리입니다.
import pandas as pd                                    # 데이터를 쉽게 보고 사용하기 위한 라이브러리입니다.

from google.colab import files                         # 구글 콜랩용 라이브러리에서 파일을 다루는 클래스입니다.

from sklearn.cluster import AgglomerativeClustering    # 사용할 머신러닝 알고리즘입니다.
from sklearn.metrics import silhouette_score           # 모델 평가를 위한 메트릭입니다.

In [2]:
# 로컬PC에서 데이터를 현재 구글 콜랩 페이지에 업로드합니다.
uploaded = files.upload()

Saving iris.csv to iris.csv


In [0]:
# 업로드한 파일 이름을 명시합니다.
## 주의: 업로드한 파일명을 확인해주세요.
filename = 'iris.csv'

# CSV 파일을 데이터프레임 형태로 가져옵니다.
## 주의: 파일 확장자를 확인해주세요.
df = pd.read_csv(io.BytesIO(uploaded[filename]), sep=',', encoding='utf-8')

In [0]:
# 4번째 컬럼까지의 데이터를 독립변수 x로 지정합니다.
## 주의: 독립변수의 컬럼범위는 데이터마다 다를 수 있습니다.
x = df.iloc[:, :4].values

# 컬럼명으로 종속변수 y를 지정합니다.
## 주의: 종속변수의 컬럼명은 데이터마다 다를 수 있습니다.
y = df['target'].values

In [0]:
# 랜덤함수의 값을 고정합니다.
## 랜덤값을 고정하여 같은 상황에서 같은 결과를 얻기 위한 것입니다. (재현성 참조)
seed = 42

In [7]:
# 생성할 클러스터 개수를 결정합니다.
n_clusters = 3

# 사용할 알고리즘을 불러옵니다.
model = AgglomerativeClustering(
    n_clusters=n_clusters,    # 나눌 클러스터의 개수를 결정합니다.
    affinity='euclidean',     # Linkage의 계산 메트릭을 결정합니다.
    linkage='ward'            # Linkage 기준을 결정합니다.
)

# 독립변수 데이터 x로 모델을 학습합니다.
## 주의: 비지도학습 모델이기 때문에 종속변수 데이터 y를 사용하지 않습니다.
model.fit(x)

AgglomerativeClustering(affinity='euclidean', compute_full_tree='auto',
                        connectivity=None, distance_threshold=None,
                        linkage='ward', memory=None, n_clusters=3)

In [0]:
# 클러스터링 결과를 구합니다.
## 주의: 클러스터는 주어진 클래스와 이름이 다를 수 있습니다.
pred = model.labels_

In [9]:
# 모델의 성능을 평가합니다.
## 실루엣 값은 한 클러스터 안의 데이터가 다른 클러스터와 비교해서 얼마나 비슷한지를 나타냅니다. 1에 가까울수록 좋습니다.
score = silhouette_score(
    X=x,
    labels=pred,
    metric='euclidean',    # 평가 메트릭을 결정합니다.
    sample_size=None,      # 샘플링 크기를 결정합니다.
    random_state=seed      # 랜덤함수로 추출할 초기값을 seed로 고정합니다.
)
print('Silhouette score: ', score)

Silhouette score:  0.5543236611296426


In [0]:
# 클러스터와 클래스의 이름을 맞춰줍니다.

# 클러스터와 클래스의 페어를 저장할 빈 리스트를 생성합니다.
pair_labels = []

# 루프(loop)를 돌며 페어를 저장합니다.
## 주의1: 루프의 수는 n_clusters의 값과 같습니다.
## 주의2: 페어의 왼쪽이 클러스터, 오른쪽이 클래스입니다.
for pred_cluster in range(n_clusters):
    cls = np.argmax(np.bincount(y[model.labels_==pred_cluster]))
    pair_labels.append((pred_cluster, cls))

# 페어 리스트를 딕셔너리(dictionary)로 형식을 바꿉니다.
pair_labels = dict(pair_labels)

# Pandas 데이터프레임에 클러스터링 결과를 저장합니다.
df['cluster'] = pred

# 페어 딕셔너리를 기준으로 cluster 컬럼의 값을 변경합니다.
df['cluster'] = df['cluster'].replace(pair_labels)

In [11]:
# 모델의 성능을 평가합니다.
## 주의: y가 주어져 있을 때 가능한 것으로 y가 없다면 비교대상이 없어 이렇게 평가할 수 없습니다.
print('Accuracy: ', len(df[df['target'] == df['cluster']].index)/len(df.index))

Accuracy:  0.8933333333333333
